# 转置卷积
:label:`sec_transposed_conv`

到目前为止我们所见过的CNN层，例如卷积层（:numref:`sec_conv_layer`）和池化层（:numref:`sec_pooling`），通常会减少（下采样）输入的空间维度（高度和宽度），或者保持它们不变。在像素级分类的语义分割中，如果输入和输出的空间维度相同将非常方便。例如，一个输出像素的通道维度可以保存同一空间位置输入像素的分类结果。

为了实现这一点，尤其是在CNN层减少了空间维度之后，我们可以使用另一种类型的CNN层来增加（上采样）中间特征图的空间维度。在本节中，我们将介绍*转置卷积*，也称为*分数步长卷积* :cite:`Dumoulin.Visin.2016`，用于通过卷积操作逆转下采样操作。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 基本操作

暂时忽略通道，
让我们从
步幅为1且无填充的基本转置卷积操作开始。
假设我们有一个
$n_h \times n_w$ 的输入张量
和一个 $k_h \times k_w$ 的内核。
以步幅为1在每行滑动内核窗口
$n_w$ 次
并在每列滑动
$n_h$ 次
产生
总共 $n_h n_w$ 个中间结果。
每个中间结果是
一个 $(n_h + k_h - 1) \times (n_w + k_w - 1)$
初始化为零的张量。
为了计算每个中间张量，
输入张量中的每个元素
与内核相乘
使得生成的 $k_h \times k_w$ 张量
替换每个中间张量的一部分。
请注意
每个中间张量中被替换部分的位置
对应于用于计算的输入张量中的元素位置。
最后，所有中间结果
相加得到输出。

例如，
:numref:`fig_trans_conv` 说明了
如何使用 $2\times 2$ 内核对 $2\times 2$ 输入张量进行转置卷积计算。


![Transposed convolution with a $2\times 2$ kernel. The shaded portions are a portion of an intermediate tensor as well as the input and kernel tensor elements used for the  computation.](../img/trans_conv.svg)
:label:`fig_trans_conv`


我们可以（**实现这个基本的转置卷积操作**）`trans_conv` 对输入矩阵 `X` 和内核矩阵 `K` 进行操作。

In [2]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

与常规卷积（在 :numref:`sec_conv_layer` 中）通过核*减少*输入元素不同，转置卷积通过核*广播*输入元素，从而产生比输入更大的输出。我们可以从 :numref:`fig_trans_conv` 构建输入张量 `X` 和核张量 `K` 来[**验证上述基本二维转置卷积操作的实现输出**]。

In [3]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

或者，当输入 `X` 和核 `K` 都是四维张量时，我们可以[**使用高级API来获得相同的结果**]。

In [4]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

## [**填充、步幅和多通道**]

不同于常规卷积中
对输入应用填充，
在转置卷积中
是对输出应用填充。
例如，
当指定高度和宽度两侧的
填充数为1时，
转置卷积输出的第一行和最后一行、
第一列和最后一列将被移除。

In [5]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

在转置卷积中，步幅是为中间结果（因此也是输出）指定的，而不是为输入指定的。使用:numref:`fig_trans_conv`中的相同输入和核张量，将步幅从1改为2会增加中间张量的高度和宽度，因此输出张量如:numref:`fig_trans_conv_stride2`所示。

![带有$2\times 2$核的转置卷积，步幅为2。阴影部分是中间张量的一部分以及用于计算的输入和核张量元素。](../img/trans_conv_stride2.svg)
:label:`fig_trans_conv_stride2`



以下代码片段可以验证:numref:`fig_trans_conv_stride2`中步幅为2的转置卷积输出。

In [6]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

对于多个输入和输出通道，转置卷积的工作方式与常规卷积相同。假设输入有 $c_i$ 个通道，并且转置卷积为每个输入通道分配一个 $k_h\times k_w$ 的核张量。当指定多个输出通道时，每个输出通道将有一个 $c_i\times k_h\times k_w$ 的核。

总之，如果我们把 $\mathsf{X}$ 输入到卷积层 $f$ 中以输出 $\mathsf{Y}=f(\mathsf{X})$，并创建一个除了输出通道数等于 $\mathsf{X}$ 的通道数之外其他超参数都与 $f$ 相同的转置卷积层 $g$，那么 $g(Y)$ 将具有与 $\mathsf{X}$ 相同的形状。这可以通过以下示例来说明。

In [7]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

## [**与矩阵转置的联系**]
:label:`subsec-connection-to-mat-transposition`

转置卷积之所以得名，是因为它与矩阵转置有关。为了说明这一点，我们首先来看看如何使用矩阵乘法来实现卷积。在下面的例子中，我们定义了一个 $3\times 3$ 的输入 `X` 和一个 $2\times 2$ 的卷积核 `K`，然后使用 `corr2d` 函数来计算卷积输出 `Y`。

In [8]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

接下来，我们将卷积核 `K` 重写为一个包含许多零的稀疏权重矩阵 `W`。权重矩阵的形状是（$4$，$9$），其中非零元素来自卷积核 `K`。

In [9]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

将输入 `X` 按行连接成一个长度为9的向量。然后 `W` 和向量化后的 `X` 的矩阵乘法得到一个长度为4的向量。
经过重塑后，我们可以从上面的原始卷积操作中获得相同的结果 `Y`：
我们刚刚使用矩阵乘法实现了卷积。

In [10]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

同样，我们可以使用矩阵乘法来实现转置卷积。在下面的例子中，我们将上述常规卷积的 $2 \times 2$ 输出 `Y` 作为转置卷积的输入。为了通过矩阵乘法实现这一操作，我们只需要将权重矩阵 `W` 转置为新的形状 $(9, 4)$。

In [11]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

考虑通过矩阵乘法来实现卷积。
给定一个输入向量 $\mathbf{x}$
和一个权重矩阵 $\mathbf{W}$，
卷积的前向传播函数
可以通过将其输入与权重矩阵相乘
并输出一个向量
$\mathbf{y}=\mathbf{W}\mathbf{x}$ 来实现。
由于反向传播
遵循链式法则
且 $\nabla_{\mathbf{x}}\mathbf{y}=\mathbf{W}^\top$，
卷积的反向传播函数
可以通过将其输入与
转置后的权重矩阵 $\mathbf{W}^\top$ 相乘来实现。
因此，
转置卷积层
可以简单地交换卷积层的前向传播函数
和反向传播函数：
其前向传播
和反向传播函数
分别将它们的输入向量与
$\mathbf{W}^\top$ 和 $\mathbf{W}$ 相乘。

## 摘要

* 与通过内核减少输入元素的常规卷积不同，转置卷积通过内核广播输入元素，从而产生比输入更大的输出。
* 如果我们将 $\mathsf{X}$ 输入到一个卷积层 $f$ 中以输出 $\mathsf{Y}=f(\mathsf{X})$ 并创建一个除了输出通道数为 $\mathsf{X}$ 的通道数之外具有相同超参数的转置卷积层 $g$，那么 $g(Y)$ 将具有与 $\mathsf{X}$ 相同的形状。
* 我们可以使用矩阵乘法来实现卷积。转置卷积层可以简单地交换卷积层的前向传播函数和反向传播函数。

## 练习

1. 在 :numref:`subsec-connection-to-mat-transposition` 中，卷积输入 `X` 和转置卷积输出 `Z` 具有相同的形状。它们的值也相同吗？为什么？
1. 使用矩阵乘法来实现卷积是否高效？为什么？

[讨论](https://discuss.d2l.ai/t/1450)